### Introduction

**Our goal is to develop a predictive model using ARIMA to forecast future temperature trends.**

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
df = pd.read_csv("/kaggle/input/climate-change-earth-surface-temperature-data/GlobalLandTemperaturesByCity.csv")
df.head(20)

### Data Cleaning

The dataset I am using contains temperature readings for Denmark over a period of time.

In [ ]:
#remove null values
df = df.dropna()
df_denmark = df[df.Country == "Denmark"]
df_denmark.index = pd.to_datetime(df_denmark.dt)
df.head()

In [ ]:
df_denmark = df_denmark.drop(['dt','AverageTemperatureUncertainty'],axis=1)
df_denmark.describe()

In [ ]:
ts = df_denmark["AverageTemperature"]
ts

#### Visualise the data

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
fig = plt.figure(1)
ax1 = fig.add_subplot(111) # 111: no of rows, cols, index
ax1.set_xlabel("date")
ax1.set_ylabel("average temperature")
ax1.plot(ts)

### The ADF (Augmented Dickey-Fuller) test for stationarity

The null hypothesis of the test is that the time series data is non-stationary.
If p-value < 5%, we reject the null hypothesis

**As we can see in the visualisation, we cannot identify a trend or seasonal component. Our next step is to check if data is stationary.**

#### Stationarity:
1. Mean is constant
2. Standard deviation is constant
3. No seasonality (ie. No predictable, periodic behavior over time)

#### White noise
1. Mean is 0
2. Standard deviation is constant
3. No seasonality 

### How to check for stationarity?
1. Visually
2. Global vs local tests 
3. ADF test


### Making a TS stationary
suppose y<sub>t</sub> = B<sub>o</sub> + B<sub>1</sub>t + e<sub>t</sub>
(e: error/white noise)

create Z<sub>t</sub> = Y<sub>t</sub> - Y<sub>t-1</sub>
                     = B<sub>1</sub> + (e<sub>t</sub>-e<sub>t-1</sub>)
                     
E(Z<sub>t</sub>)  = B<sub>1</sub> ie. constant
V(Z<sub>t</sub>)  = K<sup>2</sup> ie. constant
                     

In [ ]:

from statsmodels.tsa.stattools import adfuller
adf_result = adfuller(ts, autolag='AIC')
print(f'ADF Statistic:{adf_result[0]}')
print(f'p-value:{adf_result[1]}')

for percentage,value in adf_result[4].items():
    print("Critical Value:")
    print(f'{percentage}{value}')

**Since p-value is much smaller than 5% , we reject the null hypothesis. Hence, time series data is stationary**

### Autoregressive Moving Average (ARMA) Model

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

newmodel = ARIMA(ts, order =(1,0,1)) 
#order: (p,d,q) = autoregressive (p), differencing (d), moving average (q) 

results = newmodel.fit()
predictions = results.predict(start = '01/01/1990',end = '01/01/1991')

### Model Evaluation

In [ ]:
#to check accuracy of our ARIMA model, keeping actual values aside

actuals = df_denmark['01/01/1990':'01/01/1991']['AverageTemperature'][0:13]

In [ ]:
# Mean Absolute error: calculates the avg absolute diff between prediction and actuals

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(actuals[0:13],predictions)
print(f'MAE:{mae}')

### Model Tuning

In [ ]:
import itertools
import warnings
from statsmodels.tsa.arima.model import ARIMA

warnings.filterwarnings('ignore')

# parameter ranges
p = range(0, 4)
d = 0 #ARMA model
q = range(0, 4)

# all possible combinations of p and q
pq = itertools.product(p, q)

best_aic = float('inf') 
best_order = None

for order in pq:
    try:
        model = ARIMA(ts, order=(*order, d))
        results = model.fit()
        aic = results.aic
        if aic < best_aic:
            best_aic = aic
            best_order = order
        print(f'ARMA{order} - AIC: {aic:.2f}')
    except:
        continue

print(f'Best ARMA model: ARMA{best_order} - AIC: {best_aic:.2f}')


After grid search for ARIMA model parameters, we see that the mean absolute error decreases from 1.8 to 1.3

In [ ]:
newmodel = ARIMA(ts, order =(3,0,0)) 
#order: (p,d,q) = autoregressive (p), differencing (d), moving average (q) 

results = newmodel.fit()
predictions = results.predict(start = '01/01/1990',end = '01/01/1991')

In [ ]:
predictions = results.predict(start = '01/01/1990',end = '01/01/1991')
mae = mean_absolute_error(actuals[0:13],predictions)
print(f'MAE:{mae}')

> Thank you for reading my notebook! I hope you found it helpful in some way.
> 
> If you did, please consider upvoting it. If you have any suggestions or improvements, please leave a comment and let me know. Your feedback is greatly appreciated!